# Interoperability with Python using Pybind11

In this tutorial, we will explore how to use Pybind11 to seamlessly integrate C++ code with Python. Pybind11 is a lightweight header-only library that exposes C++ types in Python and vice versa, mainly to create Python bindings of existing C++ code. The topics we will cover include:

1. Calling Python Functions from C++
2. Using Python Types in C++
3. Converting Between Python and C++ Types

Let's dive in!

## 1. Calling Python Functions from C++

To call Python functions from C++ using Pybind11, you'll need to perform the following steps:

1. **Initialize the Python interpreter** using `Py_Initialize();`.
2. **Import the required Python module**. This can be done using the `py::module_::import("module_name")` function.
3. **Call the desired Python function**. Once you have the module, you can call any function from it just like you would in Python.

Here's a simple example where we call a Python function that adds two numbers from a C++ program:

In [ ]:
```cpp
#include <pybind11/pybind11.h>
#include <pybind11/embed.h>

namespace py = pybind11;

int main() {
    py::scoped_interpreter guard{};  // Initialize the Python interpreter

    py::object sum_func = py::module_::import("builtins").attr("sum");
    py::list numbers = {1, 2, 3, 4, 5};
    int result = sum_func(numbers).cast<int>();

    std::cout << "The sum is: " << result << std::endl;

    return 0;
}
```

In the above code, we initialize the Python interpreter, import the built-in `sum` function, and then call it with a list of numbers. The result is then cast to an `int` and printed out.

## 2. Using Python Types in C++

Pybind11 allows you to seamlessly use Python types in your C++ code. This is especially useful when you need to manipulate Python objects or when interfacing with Python libraries that return Python objects.

Here are some common Python types and how you can use them in C++ with Pybind11:

In [ ]:
```cpp
#include <pybind11/pybind11.h>

namespace py = pybind11;

int main() {
    // Using Python lists
    py::list pyList = {1, 2, 3, 4, 5};
    pyList.append(6);

    // Using Python dictionaries
    py::dict pyDict;
    pyDict["key"] = "value";
    std::string value = pyDict["key"].cast<std::string>();

    // Using Python tuples
    py::tuple pyTuple = py::make_tuple(1, "two", 3.0);
    int firstElement = pyTuple[0].cast<int>();

    return 0;
}
```

In this example, we demonstrate how to use Python lists, dictionaries, and tuples in C++ using Pybind11. The operations are intuitive and mirror the operations you would perform in Python.

## 3. Converting Between Python and C++ Types

One of the strengths of Pybind11 is its ability to automatically convert between Python and C++ types. This makes it easy to pass data back and forth between the two languages without manual serialization or deserialization.

The `cast` method provided by Pybind11 is used to convert Python objects to C++ types and vice versa. Here's how you can use it:

In [ ]:
```cpp
#include <pybind11/pybind11.h>

namespace py = pybind11;

int main() {
    // Convert Python int to C++ int
    py::object pyInt = py::int_(42);
    int cppInt = pyInt.cast<int>();

    // Convert C++ string to Python string
    std::string cppString = "Hello, Pybind11!";
    py::str pyString = py::cast(cppString);

    return 0;
}
```

In this example, we demonstrate how to convert a Python integer to a C++ integer and a C++ string to a Python string using the `cast` method.

## Real-World Scenario: Image Processing with Python and C++

Consider a situation where you're developing a high-performance application in C++ that processes vast amounts of image data. While C++ offers the speed and efficiency you need, Python boasts powerful image processing libraries like OpenCV and Pillow, providing a wide range of functionalities.

In a scenario where you wish to apply a specific image filter available in Python but time-consuming to implement in C++ from scratch, it's efficient to call the Python function directly from the C++ application.

### Python Code
We'll utilize the Pillow library to apply a Gaussian blur filter to an image.

```python
# image_processing.py

from PIL import Image, ImageFilter

def apply_gaussian_blur(input_path, output_path):
    with Image.open(input_path) as img:
        blurred = img.filter(ImageFilter.GaussianBlur(5))
        blurred.save(output_path)
```

### C++ Code
Using Pybind11, we'll call the `apply_gaussian_blur` function from the Python script.

```cpp
#include <pybind11/pybind11.h>
#include <pybind11/embed.h>

namespace py = pybind11;

int main() {
    py::scoped_interpreter guard{};  // Initialize the Python interpreter

    // Import the image_processing module
    py::object image_processing = py::module_::import("image_processing");

    // Call the apply_gaussian_blur function
    std::string input_path = "path_to_input_image.jpg";
    std::string output_path = "path_to_output_image.jpg";
    image_processing.attr("apply_gaussian_blur")(input_path, output_path);

    std::cout << "Image processing complete!" << std::endl;

    return 0;
}
```

### Explanation
In the Python script, we define a function `apply_gaussian_blur` that takes an input image path, applies a Gaussian blur filter using the Pillow library, and saves the processed image to the specified output path.

In the C++ code, we initialize the Python interpreter, import the `image_processing` module, and then directly call the `apply_gaussian_blur` function from C++, passing the paths to the input and output images.

This method allows us to harness the simplicity and power of Python libraries while retaining the performance advantages of a C++ application. It's an ideal strategy when you want to integrate Python's specialized functionalities into a C++ project without compromising on speed and efficiency.

## Real-World Scenario: Image Processing

Let's consider a practical scenario where converting between Python and C++ types proves beneficial:

### Scenario Description:

Imagine having a C++ library that performs efficient image processing operations, and you wish to utilize this library within a Python application. The C++ library accepts an image (represented as a 2D array of pixel values) and applies a filter to it. However, in the Python application, the popular `numpy` library is used for image handling and manipulation.

To bridge this gap, Pybind11 can be employed to convert `numpy` arrays to C++ arrays and vice versa. The steps involved are:

1. **Python Side**: Utilize `numpy` for image loading and handling.
2. **C++ Side**: Execute the image processing operation.
3. **Python Side**: Display the processed image.

Let's delve into the code example for this scenario.

### C++ Code (image_processing.cpp):

```cpp
#include <pybind11/pybind11.h>
#include <pybind11/numpy.h>

namespace py = pybind11;

// Simple image inversion filter
py::array_t<int> invert_image(py::array_t<int> input_image) {
    py::buffer_info buf_info = input_image.request();
    auto ptr = static_cast<int *>(buf_info.ptr);

    int X = buf_info.shape[0];
    int Y = buf_info.shape[1];

    // Create a new numpy array for the output
    py::array_t<int> output_image = py::array_t<int>(buf_info.size);
    py::buffer_info out_buf_info = output_image.request();
    auto out_ptr = static_cast<int *>(out_buf_info.ptr);

    // Invert the image
    for (size_t idx = 0; idx < buf_info.size; idx++) {
        out_ptr[idx] = 255 - ptr[idx];
    }

    output_image.resize({X, Y});

    return output_image;
}

PYBIND11_MODULE(image_processing, m) {
    m.def("invert_image", &invert_image, "Invert the input image");
}
```

#### Explanation:

1. **C++ Side**:
   - A function named `invert_image` is defined, which takes a `numpy` array (representing our image) as input.
   - The buffer information of the `numpy` array is accessed to obtain the raw data and dimensions.
   - A new `numpy` array is created for the output image.
   - The image inversion operation is executed by subtracting each pixel value from 255.
   - Finally, the inverted image is returned.

### Python Code:

```python
import numpy as np
import matplotlib.pyplot as plt
import image_processing  # Our C++ module

# Load an image using numpy (for simplicity, let's use a grayscale image)
image = np.array([[100, 150], [200, 50]])

# Use our C++ function to invert the image
inverted_image = image_processing.invert_image(image)

# Display the original and inverted images
plt.subplot(1, 2, 1)
plt.imshow(image, cmap='gray')
plt.title('Original Image')

plt.subplot(1, 2, 2)
plt.imshow(inverted_image, cmap='gray')
plt.title('Inverted Image')

plt.show()
```

#### Explanation:

1. **Python Side**:
   - A simple grayscale image is loaded using `numpy`.
   - The C++ function `invert_image` is invoked to obtain the inverted image.
   - Using `matplotlib`, the original and inverted images are displayed side by side.

This example illustrates how Pybind11 facilitates the integration of efficient C++ operations (like image processing) into a Python workflow. The seamless conversion between Python and C++ types ensures smooth and efficient interoperability between the two languages.